In [155]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import shutil
from datetime import datetime
from tabulate import tabulate

In [19]:
dirs = r'D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies'

In [167]:
#Globbing Energy Diffs and other parameters
dif = glob.glob(os.path.join(dirs,'*','Dif*.fxout'), recursive=True) #shows difference between WT and mutant counterpart
raw = glob.glob(os.path.join(dirs,'*','Raw*.fxout'), recursive=True) #shows raw energy decompositions
avg = glob.glob(os.path.join(dirs,'*','Average*.fxout'), recursive=True) #avg energy between runs
dif

['D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\C478K\\Dif_C478KMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\D440K\\Dif_D440KMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\L484A\\Dif_L484AMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\P477N\\Dif_P477NMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\P505H\\Dif_P505HMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\T498R\\Dif_T498RMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\V501Y\\Dif_V501YMMod_omicron_b11529_7cn8_RBD.fxout',
 'D:\\Documents\\Datasets\\Output\\FoldX\\1.1.4Output_Energies\\Y493R\\Dif_Y493RMMod_omicron_b11529_7cn8_RBD.fxout']

In [124]:
#converting the FXOUT file into a pandas readable format 
for files in dif:
    f = open(files,"r")
    create_file = open(os.path.join(os.path.dirname(files),f"{os.path.basename(files).replace('.fxout','.tsv')}"),"a")
    for line in f.readlines()[8:]:
        create_file.write(line)
    create_file.close()
    f.close()
    print('Done!', files, sep=':')

Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\C478K\Dif_C478KMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\D440K\Dif_D440KMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\L484A\Dif_L484AMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\P477N\Dif_P477NMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\P505H\Dif_P505HMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\T498R\Dif_T498RMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\V501Y\Dif_V501YMMod_omicron_b11529_7cn8_RBD.fxout
Done!:D:\Documents\Datasets\Output\FoldX\1.1.4Output_Energies\Y493R\Dif_Y493RMMod_omicron_b11529_7cn8_RBD.fxout


In [121]:
#Deleting the files (first run encountered an error)
# all_tsv = glob.glob(os.path.join(dirs, "*","*.tsv"), recursive = True)
# for file in all_tsv:
#     os.remove(file)

In [132]:
#Loading all TSV files created and save those in a txt file. Please check the designated file (flist and fsum) in the folder
out = glob.glob(os.path.join(dirs, "*","Dif*.tsv"), recursive = True)
mlist = []

for files in out:
    df = pd.read_csv(files, sep='\t').sort_values(by=['total energy'], ascending=True)
    mlist.append(df.iloc[0].loc['Pdb'])
    
    #save a list of all protein information with least energy changes
    flist = open(os.path.join(dirs, 'flist_sorted_out.txt'),'a')
    fsum = open(os.path.join(dirs, 'fsum_sorted_out.txt'),'a')

    flist.write(str(df.iloc[0]))
    flist.write(f"\n{50*'-'}\n")

    fsum.write(f'\nSUMMARY Sorted: Energy Differences between Generated Mutant Model and Template ({os.path.basename(files)})\n')
    fsum.write(f"Introduced mutation(s): {os.path.basename(files).split('_')[1].replace('MMod','')}\n")
    fsum.write(tabulate(df, headers='keys', tablefmt='psql'))
    fsum.write('\n')

flist.write('\n----------END----------\n')    
flist.write(str(datetime.now()))
flist.write("\nRun by: JAS")
flist.close()

fsum.write('\n----------END----------\n')    
fsum.write(str(datetime.now()))
fsum.write("\nRun by: JAS")
fsum.close()

In [166]:
#saving the most energy favored output in foldX
root = r"D:\Documents\Datasets\Output\FoldX"
folders = glob.glob(os.path.join(root,'1.1.3*','*[!.txt]'),recursive=True)
destination_folder = os.path.join(root, '1.1.5Prot_List_Output')
if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)
for fs, prots in zip(folders, mlist):
    #copy and rename
    shutil.copy(os.path.join(fs, prots), destination_folder)
    os.rename(os.path.join(destination_folder, prots),
              os.path.join(destination_folder,prots.replace('7cn8',os.path.basename(fs))))